<a href="https://colab.research.google.com/github/amornpkr/amornpkr/blob/main/Thailand_floods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/giswqs/notebook-share/blob/master/docs/pakistan_floods.ipynb)
[![Open in SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/giswqs/notebook-share/blob/master/docs/pakistan_floods.ipynb)
[![Open in Planetary Computer](https://img.shields.io/badge/Open-Planetary%20Computer-black?style=flat&logo=microsoft)](https://pccompute.westeurope.cloudapp.azure.com/compute/hub/user-redirect/git-pull?repo=https://github.com/giswqs/notebook-share&urlpath=lab/tree/notebook-share/docs/pakistan_floods.ipynb&branch=master)
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/giswqs/notebook-share/HEAD?labpath=docs%pakistan_floods.ipynb)

## การติดตามพื้นที่น้ำท่วมของประเทศไทยปี 2565 โดยใช้ Earth Engine ไลบรารี่ Geemap



## บทนำ

สถานการณ์น้ำท่วมในภาคตะวันออกเฉียงเหนือของประเทศไทยในปี 2565 มีพื้นที่ถูกน้ำท่วมประมาณ 3.96 ล้านไร่ รวม 20 จังหวัด 271 อำเภอ 1,760 ตำบล ได้แก่ จังหวัดนครราชสีมา ร้อยเอ็ด สุรินทร์ ศรีสะเกษ บุรีรัมย์ มหาสารคาม ขอนแก่น อุบลราชธานี ชัยภูมิ ยโสธร กาฬสินธุ์ (https://www.thaiwater.net/)


## วิธีการวิเคราะห์

แนวทางการการใช้ไฟล์ Coding นี้ อันดับแรกท่านต้องลงทะเบียนเมลล์ของ Gmail ก่อน แล้วเข้าไปทำการยืนยันตัวตนผ่าน Google Authenticate หลังจากนั้นให้ท่านเข้าไปที่ Colab.reseach.google.com

ตัวอย่างของการใช้ Jupyter Notebook นี้ ภายใต้ไลบรารี่ Geemap ท่านต้องติดตั้งไลบรารี่ก่อนทุกครั้ง โดยใช้ !pip install geemap


## การติดตั้งไลบารี Geemap

In [ ]:
!pip install geemap

##ใช้คำสั่งนำเข้าไลบรารีพื้นฐานก่อน

นำเข้า earthengine-api และ geemap.

In [3]:
import ee
import geemap

## สร้างแผนที่ Web เพื่อแสดงพื้นที่ศึกษา

ระบบตำแหน่งทางพิกัดภูมิศาสตร์`[lat, lon]` ระดับการซูมให้กับแผนที่.

In [10]:
Map = geemap.Map(center=[15.212646, 104.829613], zoom=12)
Map

Map(center=[15.212646, 104.829613], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBo…

## การสืบค้นข้อมูล

คลิกที่ไอคอน global ด้านซ้ายบนของแผนที่เพื่อสืบค้นข้อมูล ดังตัวอย่าง และพิมพ์เพื่อค้นหาข้อมูล เช่น 'countries' แล้วกด Enter ตัวอย่างเป็นข้อมูล [`LSIB 2017: Large Scale International Boundary Polygons, Simplified`](https://developers.google.com/earth-engine/datasets/catalog/USDOS_LSIB_SIMPLE_2017). คลิกที่ `import` เพื่อเพิ่ม Code มาวางไว้ที่ Jupyter Notebook.

![](https://i.imgur.com/1ef0sDz.gif)

ในตัวอย่างนี้ เราจะชี้เป้าพื้นที่บริเวณจังหวัดอุบลราชธานี ลุ่มน้ำชี Code จะแสดงและวิเคราะห์พื้นที่น้ำท่วมอย่างง่าย.

In [14]:
dataset = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
styleParams = {
  'fillColor': '00000000',
  'color': 'ff0000',
  'width': 3.0,
}
countries = dataset.style(**styleParams)
Map.setCenter(104.829613,15.212646, 12)
Map.addLayer(countries, {}, 'USDOS/LSIB_SIMPLE/2017')


In [25]:
country_name = 'Thailand'
pre_flood_start_date = '2021-09-01'
pre_flood_end_date = '2021-11-30'
flood_start_date = '2022-09-01'
flood_end_date = '2022-11-30'

## การแสดงผลข้อมูล


In [26]:
Map = geemap.Map()

country = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(
    ee.Filter.eq('country_na', country_name)
)

style = {'color': 'black', 'fillColor': '00000000'}
Map.addLayer(country.style(**style), {}, country_name)
Map.centerObject(country)

Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## การสร้างภาพสีผสมจากข้อมูลดาวเทียม Landsat
สร้างภาพสีผสมจากข้อมูลดาวเทียม Landsat เพื่อเปรียบเทียมน้ำท่วมเดือนกันยายน - ตุลาคม ปี 2564 และน้ำท่วมในปี 2565 ข้อมูลที่ใช้จะอยู่ Product ข้อมูล [USGS Landsat 8 Collection 2 Tier 1 Raw Scenes](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1).

In [27]:
Map = geemap.Map()

landsat_col_2021 = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1')
    .filterDate(pre_flood_start_date, pre_flood_end_date)
    .filterBounds(country)
)
landsat_2021 = ee.Algorithms.Landsat.simpleComposite(landsat_col_2021).clipToCollection(
    country
)

vis_params = {'bands': ['B6', 'B5', 'B4'], 'max': 128}
Map.addLayer(landsat_2021, vis_params, 'Landsat 2021')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

Create a Landsat 8 composite for the flood period (August 1 to September 30, 2022).

In [28]:
landsat_col_2022 = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1')
    .filterDate(flood_start_date, flood_end_date)
    .filterBounds(country)
)
landsat_2022 = ee.Algorithms.Landsat.simpleComposite(landsat_col_2022).clipToCollection(
    country
)

Map.addLayer(landsat_2022, vis_params, 'Landsat 2022')
Map.centerObject(country)
Map

Map(bottom=30304.0, center=[15.008463695004872, 101.10717773437501], controls=(WidgetControl(options=['positio…

## เปรียบข้อมูลดาวเทียม Landsat-8 ปี 2564 และ 2565



In [31]:
Map = geemap.Map()
Map.setCenter(104.710034, 15.171696, 12),

left_layer = geemap.ee_tile_layer(landsat_2021, vis_params, 'Landsat 2021')
right_layer = geemap.ee_tile_layer(landsat_2022, vis_params, 'Landsat 2022')

Map.split_map(
    left_layer, right_layer, left_label='Landsat 2021', right_label='Landsat 2022'
)
Map.addLayer(country.style(**style), {}, country_name)
Map

Map(center=[15.171696, 104.710034], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

## ค่าดัชนีผลต่างความชื้นของนํ้า (NDWI)

ค่า Normalized Difference Water Index(https://en.wikipedia.org/wiki/Normalized_difference_water_index) (NDWI) คือ ดัชนีความแตกต่างของนํ้าปกติ. การตรวจวัดระดับความชื้นในดินหรือพืชพรรณจากการสะท้อนรังสีของดวงอาทิตย์ในสัดส่วนที่แตกต่างกันของแต่ละช่วงคลื่น หากมีปริมาณน้ำในดินหรือพืชพรรณมาก จะทําให้รังสีในช่วงคลื่นอินฟาเรดสั้น (SWIR) ถูกดูดซับมาก ส่งผลให้มีการสะท้อนรังสีออกมาน้อยลง ทำให้ค่าดัชนี NDWI ที่คํานวณได้มี ค่าสูงขึ้นตามไปด้วยเป็นอีกเทคนิคหนึ่งในการตรวจวัดหาพื้นที่ภัยแล้งโดยอ้างอิงจากค่าความชื้นในดินและพืช
 ใช้สมการ:

$$NDWI = \frac{Green - NIR}{Green + NIR}$$

ขณะที่ช่วงคลื่น Green  และ NIR คือ ช่วงคลื่นของข้อมูลดาวเทียม NDWI values มีค่าระหว่าง -1 to 1. ค่า NDWI values มักจะเป็นค่าบวก (ระหว่าง 0.1-0.3) จะเป็นพื้นที่แหล่งน้ำ

ช่วงคลื่นที่ใช้ใน Landsat 8 [11 spectral bands](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1#bands). Landsat 8 NDWI จะใช้ช่วงคลื่นที่ (`B3`) และ NIR (`B5`) bands.

![](https://i.imgur.com/yuZthc6.png)

In [32]:
ndwi_2021 = landsat_2021.normalizedDifference(['B3', 'B5']).rename('NDWI')
ndwi_2022 = landsat_2022.normalizedDifference(['B3', 'B5']).rename('NDWI')

คำนวณหาค่า NDWI จากภาพปี 2564 และ 2565.

In [33]:
Map = geemap.Map()
Map.setCenter(104.710034, 15.171696, 12)

ndwi_vis = {'min': -1, 'max': 1, 'palette': 'ndwi'}

left_layer = geemap.ee_tile_layer(ndwi_2021, ndwi_vis, 'NDWI 2021')
right_layer = geemap.ee_tile_layer(ndwi_2022, ndwi_vis, 'NDWI 2022')

Map.split_map(left_layer, right_layer, left_label='NDWI 2021', right_label='NDWI 2022')
Map.addLayer(country.style(**style), {}, country_name)
Map

Map(center=[15.171696, 104.710034], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

## คำนวณหาขอบเขตพื้นที่น้ำท่วมจากข้อมูลดาวเทียม

การคำนวณหาค่าพื้นที่น้ำท่วม จำเป็นจะต้องแปลงข้อมูล NDWI เป็น Binary image แล้วดึงค่าที่เป็นพื้นที่น้ำออกมา ค่าที่ดึงออกมาอยู่ระหว่าง 0.1 - 0.3 ซึ่งเป็นค่าที่แคบมาก โดยส่วนใหญ่พื้นที่น้ำจะอยู่ในช่วงนี้ ซึ่งเป็นค่าบวก

In [34]:
threshold = 0.1
water_2021 = ndwi_2021.gt(threshold)
water_2022 = ndwi_2022.gt(threshold)

การรวมข้อมูลน้ำท่วม 2 ชุดเข้าด้วยกัน

In [36]:
Map = geemap.Map()
Map.setCenter(104.710034, 15.171696, 12)

Map.addLayer(landsat_2021, vis_params, 'Landsat 2021', False)
Map.addLayer(landsat_2022, vis_params, 'Landsat 2022', False)

left_layer = geemap.ee_tile_layer(
    water_2021.selfMask(), {'palette': 'blue'}, 'Water 2021'
)
right_layer = geemap.ee_tile_layer(
    water_2022.selfMask(), {'palette': 'red'}, 'Water 2022'
)

Map.split_map(
    left_layer, right_layer, left_label='Water 2021', right_label='Water 2022'
)
Map.addLayer(country.style(**style), {}, country_name)
Map

Map(center=[15.171696, 104.710034], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

## การสร้างแผนที่น้ำท่วมระหว่าง 2 ช่วงเวลา

เพื่อแยกระดับน้ำที่ท่วม เราจำเป็นต้องลบระดับน้ำก่อนน้ำท่วมออกจากระดับน้ำที่ท่วมปัจจุบัน ระดับน้ำท่วม คือ ความแตกต่างระหว่างระดับน้ำท่วมปีปจจุบันและระดับน้ำก่อนน้ำท่วม กล่าวอีกนัยหนึ่ง ค่าพิกเซลที่ระบุว่าเป็นน้ำในช่วงน้ำท่วม แต่ไม่ได้อยู่ในช่วงก่อนน้ำ จะถือว่าเป็นพิกเซลที่ถูกน้ำท่วม เราใช้คำสั่ง `selfMask()` ใช้เพื่อปกปิดพิกเซลที่ไม่มีข้อมูล

In [39]:
flood_extent = water_2022.subtract(water_2021).gt(0).selfMask()

เพิ่มขอบเขตน้ำท่วมในแผนที่

In [38]:
Map = geemap.Map()
Map.setCenter(104.710034, 15.171696, 12)

Map.addLayer(landsat_2021, vis_params, 'Landsat 2021', False)
Map.addLayer(landsat_2022, vis_params, 'Landsat 2022', False)

left_layer = geemap.ee_tile_layer(
    water_2021.selfMask(), {'palette': 'blue'}, 'Water 2021'
)
right_layer = geemap.ee_tile_layer(
    water_2022.selfMask(), {'palette': 'red'}, 'Water 2022'
)

Map.split_map(
    left_layer, right_layer, left_label='Water 2021', right_label='Water 2022'
)

Map.addLayer(flood_extent, {'palette': 'cyan'}, 'Flood Extent')
Map.addLayer(country.style(**style), {}, country_name)
Map

Map(center=[15.171696, 104.710034], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

### การคำนวณพื้นที่น้ำท่วม

In [40]:
area_2021 = geemap.zonal_stats(
    water_2021.selfMask(), country, scale=1000, statistics_type='SUM', return_fc=True
)
geemap.ee_to_df(area_2021)

Computing statistics ...


,sum,wld_rgn,country_na,abbreviati,country_co
0,2972.768627,SE Asia,Thailand,Thai.,TH


In [41]:
area_2022 = geemap.zonal_stats(
    water_2022.selfMask(), country, scale=1000, statistics_type='SUM', return_fc=True
)
geemap.ee_to_df(area_2022)

Computing statistics ...


,sum,wld_rgn,country_na,abbreviati,country_co
0,3439.490196,SE Asia,Thailand,Thai.,TH


In [42]:
flood_area = geemap.zonal_stats(
    flood_extent.selfMask(), country, scale=1000, statistics_type='SUM', return_fc=True
)
geemap.ee_to_df(flood_area)

Computing statistics ...


,sum,wld_rgn,country_na,abbreviati,country_co
0,1634.082353,SE Asia,Thailand,Thai.,TH


พื้นที่้ำท่วมประเทศไทยระหว่างปี 2564-2565 คิดเป็นพื้นที่ประมาณ 1,634 ตารางกิโลเมตร

## สร้างข้อมูลภาพจากดาวเทียมในระบบเรดาห์ Sentinel-1

นอกจากข้อมูลดาวเทียม Landsat แล้ว เรายังสามารถใช้ข้อมูล Sentinel-1 [Synthetic Aperture Radar (SAR)](https://www.earthdata.nasa.gov/learn/backgrounders/what-is-sar) เพื่อแยกระดับน้ำท่วมได้เช่นกัน ข้อมูลดาวเทียมในระบบเรดาร์สามารถรวบรวมได้จากสัญญาณในระนาบโพลาไรเซชันต่างๆ โดยการควบคุมระนาบการโพลาไรเซชันที่วิเคราะห์ทั้งในเส้นทางในแนวส่งและรับ สัญญาณที่ปล่อยออกมาในแนวตั้ง (V) และได้รับในแนวนอน (H) โพลาไรเซชันจะถูกระบุด้วย VH สัญญาณที่ปล่อยออกมาในแนวนอน (H) และได้รับในแนวนอน (H) จะถูกระบุด้วย HH เป็นต้น การตรวจสอบความแรงของสัญญาณจากระนาบโพลาไรเซชันที่แตกต่างกันเหล่านี้จะนำข้อมูลเกี่ยวกับโครงสร้างของพื้นผิวที่ถ่ายภาพ การกระเจิงสัญญาณของพื้นผิวที่ขรุขระ เช่น ที่เกิดจากดินเปล่าหรือน้ำ มีความไวต่อการกระเจิงของระนาบ VV มากที่สุด ดังนั้นจึงมักใช้โพลาไรเซชัน VV เพื่อตรวจจับแหล่งน้ำ


ผลิตภัณฑ์ข้อมูลดาวเทียม Sentinel-1 มี 5 แบบ คือ (https://sentinels.copernicus.eu/web/sentinel/missions/sentinel-1/instrument-payload):

* Stripmap (SM)
* Interferometric Wide swath (IW)
* Extra-Wide swath (EW)
* Wave mode (WV)

The Interferometric Wide swath (IW) ขนาดภาพประมาณ 250 ตร.กม. ขนาดจุดภาพ 5x20 เมตร ซึ่งเราใช้ Mode นี้ในการศึกษาพื้นที่น้ำท่วม


In [43]:
s1_col_2021 = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
    .filterDate(pre_flood_start_date, pre_flood_end_date)
    .filterBounds(country)
    .select('VV')
)

247 Sentinel-1 IW mode images are available for the flood period.

In [44]:
# s1_col_2021

Create the Sentinel-1 image collection for the flood period.

In [45]:
s1_col_2022 = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
    .filterDate(flood_start_date, flood_end_date)
    .filterBounds(country)
    .select('VV')
)

In [48]:
#s1_col_2022

สร้างภาพสีผสมของข้อมูลปี 2564 และ 2565

In [49]:
Map = geemap.Map()
Map.add_basemap('HYBRID')
sar_2021 = s1_col_2021.reduce(ee.Reducer.percentile([20])).clipToCollection(country)
sar_2022 = s1_col_2022.reduce(ee.Reducer.percentile([20])).clipToCollection(country)
Map.addLayer(sar_2021, {'min': -25, 'max': -5}, 'SAR 2021')
Map.addLayer(sar_2022, {'min': -25, 'max': -5}, 'SAR 2022')
Map.centerObject(country)
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## วิธีการกำจัดสัญญาณรบกวนออกจากจุดภาพ (speckle filtering)

Speckle หรือสัญญาณรบกวน หรือบางทีเราเรียกว่าจุดกระ ที่ปรากฏในภาพเรดาร์รูรับแสงสังเคราะห์แบบ SAR เป็นสัญญาณรบกวนแบบละเอียด เกิดจากการรบกวนของคลื่นที่สะท้อนจากสัญญาณรบกวนพื้นฐานจำนวนมาก Speckle ในภาพ SAR ทำให้มีปัญหาการตีความภาพที่มีความซับซ้อน โดยลดประสิทธิภาพของการแบ่งส่วนภาพและการจัดประเภท ([Lee et al., 1994](https://doi.org/10.1080/02757259409532206)) ดังนั้น การกรองจุดภาพจึงมักใช้ร่วมกับภาพ SAR เพื่อลดจุดรบกวน ในตัวอย่างนี้ เราใช้ตัวกรองแบบ morphology หรือจุดกรองทางกายภาพ ตัวกรองจุดทางสัณฐานวิทยาเป็นตัวกรองที่ไม่ใช่เชิงเส้นที่ใช้ค่ามัธยฐานของพิกเซลและพิกเซลข้างเคียงเพื่อแทนที่ค่าพิกเซล ขนาดเคอร์เนลตั้งไว้ที่ 100 เมตร

In [50]:
col_2021 = s1_col_2021.map(lambda img: img.focal_median(100, 'circle', 'meters'))
col_2022 = s1_col_2022.map(lambda img: img.focal_median(100, 'circle', 'meters'))

In [51]:
Map = geemap.Map()
Map.add_basemap('HYBRID')
sar_2021 = col_2021.reduce(ee.Reducer.percentile([20])).clipToCollection(country)
sar_2022 = col_2022.reduce(ee.Reducer.percentile([20])).clipToCollection(country)
Map.addLayer(sar_2021, {'min': -25, 'max': -5}, 'SAR 2021')
Map.addLayer(sar_2022, {'min': -25, 'max': -5}, 'SAR 2022')
Map.centerObject(country)
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## เปรียบเทียบภาพสีผสม ทั้ง 2 ปี


In [52]:
Map = geemap.Map()
Map.setCenter(104.710034, 15.171696, 12)

left_layer = geemap.ee_tile_layer(sar_2021, {'min': -25, 'max': -5}, 'SAR 2021')
right_layer = geemap.ee_tile_layer(sar_2022, {'min': -25, 'max': -5}, 'SAR 2022')

Map.split_map(
    left_layer, right_layer, left_label='Sentinel-1 2021', right_label='Sentinel-1 2022'
)
Map.addLayer(country.style(**style), {}, country_name)
Map

Map(center=[15.171696, 104.710034], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

## การหาขอบเขตพื้นที่น้ำท่วม

พื้นที่น้ำมักจะปรากฏเป็นสีเข้มในภาพ SAR เนื่องจากคลื่นเรดาร์จะสะท้อนต่างกันไปตามพื้นผิวที่แตกต่างกัน น้ำเป็นพื้นผิวเรียบที่สะท้อนคลื่นเรดาร์ได้ไม่ดีนัก ดังนั้นในภาพ SAR จึงดูมืด ภาพ SAR แบบ Thresholding เป็นหนึ่งในแนวทางที่ใช้กันอย่างแพร่หลายในการกำหนดขอบเขตของน้ำ ([Liang and Liu, 2020](https://doi.org/10.1016/j.isprsjprs.2019.10.017)) โดยทั่วไปวิธีการกำหนดเกณฑ์สามารถแบ่งออกได้เป็นสองประเภท: ระดัยกลางและระดับพื้นที่ วิธีการกำหนดเกณฑ์ส่วนกลางใช้ค่าเกณฑ์เดียวเพื่อแบ่งส่วนภาพทั้งหมด วิธีการกำหนดเกณฑ์เฉพาะที่ใช้ค่าเกณฑ์ที่แตกต่างกันสำหรับแต่ละพิกเซล ในตัวอย่างนี้ เราใช้วิธีการกำหนดเกณฑ์ส่วนกลางเพื่อแยกขอบเขตของน้ำ ค่าเกณฑ์ถูกกำหนดเป็น -18 เดซิเบล

In [53]:
threshold = -18
water_2021 = sar_2021.lt(threshold)
water_2022 = sar_2022.lt(threshold)

Create a split-view map to compare the pre-flood and flood water extent side by side.

In [54]:
Map = geemap.Map()
Map.setCenter(104.710034, 15.171696, 12)

Map.addLayer(sar_2021, {'min': -25, 'max': -5}, 'SAR 2021')
Map.addLayer(sar_2022, {'min': -25, 'max': -5}, 'SAR 2022')

left_layer = geemap.ee_tile_layer(
    water_2021.selfMask(), {'palette': 'blue'}, 'Water 2021'
)
right_layer = geemap.ee_tile_layer(
    water_2022.selfMask(), {'palette': 'red'}, 'Water 2022'
)

Map.split_map(
    left_layer, right_layer, left_label='Water 2021', right_label='Water 2022'
)
Map.addLayer(country.style(**style), {}, country_name)
Map

Map(center=[15.171696, 104.710034], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

## การสร้างพื้นที่น้ำท่วม

เป็นวิธีที่คล้ายกับวิธีของดาวเทียม Landsat เราสามารถลบขอบเขตของน้ำก่อนน้ำท่วมออกจากขอบเขตของน้ำท่วม เพื่อแยกขอบเขตของน้ำท่วม

In [55]:
flood_extent = water_2022.subtract(water_2021).gt(0).selfMask()

ระดับน้ำท่วม คือ ความแตกต่างระหว่างระดับน้ำท่วมและระดับน้ำก่อนน้ำท่วม กล่าวอีกนัยหนึ่ง พิกเซลที่ระบุว่าเป็นน้ำในช่วงน้ำท่วมแต่ไม่ได้อยู่ในช่วงก่อนน้ำท่วมจะถือว่าเป็นพิกเซลที่ถูกน้ำท่วม ซึ่งแสดงเป็นสีฟ้า


In [56]:
Map = geemap.Map()
Map.setCenter(104.710034, 15.171696, 12)

Map.addLayer(sar_2021, {'min': -25, 'max': -5}, 'SAR 2021')
Map.addLayer(sar_2022, {'min': -25, 'max': -5}, 'SAR 2022')

left_layer = geemap.ee_tile_layer(
    water_2021.selfMask(), {'palette': 'blue'}, 'Water 2021'
)
right_layer = geemap.ee_tile_layer(
    water_2022.selfMask(), {'palette': 'red'}, 'Water 2022'
)

Map.split_map(
    left_layer, right_layer, left_label='Water 2021', right_label='Water 2022'
)

Map.addLayer(flood_extent, {'palette': 'cyan'}, 'Flood Extent')
Map.addLayer(country.style(**style), {}, country_name)
Map

Map(center=[15.171696, 104.710034], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

## คำนวณพื้นที่น้ำจากข้อมูล SAR

In [57]:
area_2021 = geemap.zonal_stats(
    water_2021.selfMask(), country, scale=1000, statistics_type='SUM', return_fc=True
)
geemap.ee_to_df(area_2021)

Computing statistics ...


,sum,wld_rgn,country_na,abbreviati,country_co
0,3529.321569,SE Asia,Thailand,Thai.,TH


In [58]:
area_2022 = geemap.zonal_stats(
    water_2022.selfMask(), country, scale=1000, statistics_type='SUM', return_fc=True
)
geemap.ee_to_df(area_2022)

Computing statistics ...


,sum,wld_rgn,country_na,abbreviati,country_co
0,3849.541176,SE Asia,Thailand,Thai.,TH


In [59]:
flood_area = geemap.zonal_stats(
    flood_extent.selfMask(), country, scale=1000, statistics_type='SUM', return_fc=True
)
geemap.ee_to_df(flood_area)

Computing statistics ...


,sum,wld_rgn,country_na,abbreviati,country_co
0,825.47451,SE Asia,Thailand,Thai.,TH


พื้นที่น้ำท่วมระหว่างปี 2564 - 2565 คิดเป็นพื้นที่ 826 ตร.กม.